In [19]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, MetaData, ForeignKey, DECIMAL
# Carga tu CSV
metadata = MetaData()

# Crea la conexión a MySQL
usuario = "root"
contraseña = "trolldesu2"
host = "localhost"
puerto = "3306"
base_datos = "mineduc"

# Usa pymysql o mysqlclient según lo que instalaste
engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")


In [20]:
datos_egreso = Table('datos_egreso', metadata,
    Column('AGNO', Integer),
    Column('RBD', Integer),
    Column('DGV_RBD', Integer),
    Column('COD_REG_RBD', Integer),
    Column('NOM_REG_RBD_A', String(100)),
    Column('COD_PRO_RBD', Integer),
    Column('COD_COM_RBD', Integer),
    Column('NOM_COM_RBD', String(100)),
    Column('COD_DEPROV_RBD', Integer),
    Column('NOM_DEPROV_RBD', String(100)),
    Column('COD_ENSE', Integer),
    Column('COD_GRADO', Integer),
    Column('COD_DEPE', Integer),
    Column('COD_DEPE2', Integer),
    Column('RURAL_RBD', Integer),
    Column('MRUN', String(20)),
    Column('MRUN_IPE', String(20)),
    Column('PROM_NOTAS_ALU', DECIMAL(4, 2)),
    Column('ORIGEN', String(50)),
    Column('ENSE_COMPLETA', Integer),
    Column('MARCA_EGRESO', Integer)
)


In [21]:
dim_alumno = Table('dim_alumno', metadata,
    Column('id_alumno', Integer, primary_key=True, autoincrement=True),
    Column('MRUN', String(20), unique=True),
    Column('GEN_ALU', Integer),
    Column('FEC_NAC_ALU', String(10)),
    Column('EDAD_ALU', Integer)
)

dim_establecimiento = Table('dim_establecimiento', metadata,
    Column('id_establecimiento', Integer, primary_key=True, autoincrement=True),
    Column('RBD', Integer),
    Column('NOM_COM_RBD', String(100)),
    Column('COD_DEPE2', Integer),
    Column('RURAL_RBD', Integer)
)

dim_curso = Table('dim_curso', metadata,
    Column('id_curso', Integer, primary_key=True, autoincrement=True),
    Column('COD_ENSE', Integer),
    Column('COD_GRADO', Integer),
    Column('LET_CUR', String(5)),
    Column('COD_JOR', Integer)
)

dim_tiempo = Table('dim_tiempo', metadata,
    Column('id_tiempo', Integer, primary_key=True, autoincrement=True),
    Column('AGNO', Integer)
)

dim_estado_final = Table('dim_estado_final', metadata,
    Column('id_estado', Integer, primary_key=True, autoincrement=True),
    Column('SIT_FIN', String(5)),
    Column('SIT_FIN_R', String(5))
)

In [22]:
hechos_egreso = Table('hechos_egreso', metadata,
    Column('id_hecho', Integer, primary_key=True, autoincrement=True),
    Column('id_alumno', Integer, ForeignKey('dim_alumno.id_alumno')),
    Column('id_establecimiento', Integer, ForeignKey('dim_establecimiento.id_establecimiento')),
    Column('id_curso', Integer, ForeignKey('dim_curso.id_curso')),
    Column('id_tiempo', Integer, ForeignKey('dim_tiempo.id_tiempo')),
    Column('id_estado', Integer, ForeignKey('dim_estado_final.id_estado')),
    Column('ENSE_COMPLETA', Integer),
    Column('MARCA_EGRESO', Integer),
    Column('PROM_NOTAS_ALU', DECIMAL(4, 2))
)


In [23]:
metadata.drop_all(engine)
metadata.create_all(engine)

In [24]:
csv_path = 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/20230313_Notas_y_Egresados_Enseñanza_Media_2024_PUBL.csv'
df = pd.read_csv(csv_path, sep=';', dtype=str).fillna('')

In [25]:
int_cols = ['AGNO', 'RBD', 'DGV_RBD', 'COD_REG_RBD', 'COD_PRO_RBD', 'COD_COM_RBD',
            'COD_DEPROV_RBD', 'COD_ENSE', 'COD_GRADO', 'COD_DEPE', 'COD_DEPE2',
            'RURAL_RBD', 'ENSE_COMPLETA', 'MARCA_EGRESO']

float_cols = ['PROM_NOTAS_ALU']

for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
hechos_egreso.to_sql('hechos_egresados', con=engine, if_exists='append', index=False)

274669